In [1]:
#Change ClsStrt from 0/1 , cls_num , dataload , Rate

In [2]:
from augment import *
import glob
import os
import glob
import scipy
import random
import bisect
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import signal
from scipy.signal import find_peaks
import matplotlib.pyplot as plt
from preprocess import *                         #ماژول استخراج پنجره ها
from data import *                               #ماژول محلی ورود داده ها
from augment import *                            #ماژول های داده افزایی
from GAN import *                  
from lstm_cnn import *                           #ماژول های داده افزایی با lstm_cnn
import augment
import importlib
importlib.reload(augment)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import BatchNormalization
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report,recall_score,precision_score
from tensorflow.keras import callbacks
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D,Conv1D,Dropout,MaxPooling1D,MaxPooling2D,Flatten,Dense
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
cwd = os.getcwd() #

In [3]:
def dstnc(dt2,dt1): #فاصله و واریانس بردار فاصله ی هر کدام از ردیف های دیتاست(آگمنت)1 از (کل)2
    dist,vrnc,i,h,w=[],[],0,np.shape(dt1)[0],np.shape(dt1)[1]
    end=min(len(dt2[0]),len(dt1[0]))
    for i in range (h):
        ds1 = dt2-dt1[i,:end] #.reshape(-1,1) #distance of every training instance
        dist_array=(ds1*10)**3  #معیار فاصله با حساسیت به توان 3 (حفظ علامت و تاثیر بیشتر تفاوت زیاد در نقاط)
        vr=np.var(ds1)
        ds1=min(np.sum(np.absolute(dist_array),axis=1))
        #print('dist2 = ', dist2)
        vrnc.append(vr)
        dist.append(ds1)
    variance=np.reshape(vrnc,(-1,1))
    distnc=np.reshape(dist,(-1,1))
    return(distnc,variance)

In [4]:
rate=480
cls_num=5
btch=20
for i in range (1,cls_num+1):
    vars()['ecg'+str(i)],vars()['ecg_tst'+str(i)]=MixedShapesSmallTrain(i)

os.chdir(cwd)
clses_lens=np.array([])
i=0                               #جمع آوری داده ها و چاپ تعداد نمونه ی هر کلاس
ecg=np.array(ecg1)
print(1,len(vars()['ecg'+str(1)]), end='\t')
clses_lens=np.append(clses_lens,len(vars()['ecg'+str(1)]))
for i in range (2,cls_num+1):
    ecg=np.concatenate((ecg,vars()['ecg'+str(i)]),axis=0)
    clses_lens=np.append(clses_lens,len(vars()['ecg'+str(i)]))
    print(i,len(vars()['ecg'+str(i)]), end='\t')
mx_sig=max(clses_lens)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


first_train_shape= (100, 1025)
first_test_shape= (2425, 1025)
classes_quantity= 5
tr_lbls=	 {1, 2, 3, 4, 5}
Count_labels= [20 20 20 20 20]
max(train_feature_Altitude)= 5.0
min(train_feature_Altitude)= -2.8941765
first_train_sample=
 [1.        1.1309687 1.1204832 ... 1.0931087 1.1025334 1.1120469]
1 20	2 20	3 20	4 20	5 20	

In [5]:
wdth=int(len(ecg1[0]))
os.chdir(cwd)
base_wn=np.array([])
windws=np.array([])

for cls in range (1,cls_num+1):                   #ساخت پنجره های داده های آموزش اصلی
    vars()['wndws'+str(cls)]=np.array(vars()['ecg'+str(cls)])
    base_wn=np.append(base_wn,vars()['wndws'+str(cls)])
base_wn=np.reshape(base_wn,(int(len(base_wn)/wdth),wdth))  #base data windows
print("shape wndws1=", np.shape(wndws1))
print("shape base_wn=", np.shape(base_wn))

furir_windws0=np.fft.fft(np.array(base_wn[:,:-1]))
print("shape furir_windws0=", np.shape(furir_windws0))
furir_windws=np.concatenate((np.array(furir_windws0),np.transpose([base_wn[:,-1]])),axis=1)   
print("shape furir_windws=", np.shape(furir_windws))

smpl_rte=len(base_wn[0])-1                 #len(ecg1[0])  # در ماژول ها نیز همین مقدار ثبت شده
print("smpl_rte=",smpl_rte)
mx_wndws=np.max(np.bincount(np.int16(base_wn[:,-1])))
print("mx_wndws=",mx_wndws)

shape wndws1= (20, 481)
shape base_wn= (100, 481)
shape furir_windws0= (100, 480)
shape furir_windws= (100, 481)
smpl_rte= 480
mx_wndws= 20


In [6]:
path=('{}\AugEvl'.format(cwd)) #create folder 
if not os.path.isdir(path):
    os.mkdir(path)

def addqlty(base_wn_dt,data,mthd,i):               
    cls=int(i)
    c,alpha,PL=0,1,1
    #from glob import glob
    cwd = os.getcwd() #
    adrs=cwd+'/Aug/aug'+str(i)+str(mthd)+'.csv'
    #alldata = np.array([])                    # List for storing all the data
    train_labels=np.array(base_wn_dt[:,-1])
    tr=np.array(base_wn_dt[:,:-1])
    indx_pstv=np.where(base_wn_dt[:,-1] == i)
    P=np.array(base_wn_dt[indx_pstv])
    indx_ngtv=np.where(base_wn_dt[:,-1] != i)
    N=np.array(base_wn_dt[indx_ngtv])

    #data = np.array(data)
    aug=np.array(data)
    pnts=int(np.shape(data)[1])
    
    #aug=data[data[:,-2].argsort()]        # Positive data Sorted Base Distance
    trl=np.array(train_labels)
    for j in range (len(trl)):
        if train_labels[j]==i :           # !!!!! == (Paper GDO)
            trl[j]=1
        else :
            trl[j]=0
    trlt=np.array([trl]).T
    #mn_ds_pstv=np.reshape(aug[:,-2],(-1,1)) #min dis to positive class (submited in -2 column)
    #print('shape and aug[0]=',np.shape(aug[0]),aug[0])
    #print('shape P and P[0]=',np.shape(P),P[0])    
    qlty1,qlty2=mn_ds_pstv,vr_P=dstnc(P[:,1:-1],aug[:,1:-1])  #min distance to negative class
    new1=np.append(aug,qlty1,axis=1)
    new2=np.append(new1,qlty2,axis=1)
    mn_ds_ngtv,vr_N=dstnc(N[:,1:-1],aug[:,1:-1])              #min distance to negative class
    qlty3=mn_ds_pstv/mn_ds_ngtv                               #Smaller is Better
    qlty4=np.reshape(valuation(aug[:,:-1],tr,trlt),(-1,1))   #line 272
    new3=np.append(new2,qlty3,axis=1)
    new4=np.append(new3,qlty4,axis=1)
    for j in range (len(trl)):
        if train_labels[j]!=i :                          # !!!!! == (!= Inverse Paper GDO)
            trl[j]=1
        else :
            trl[j]=0
    trlt=np.array([trl]).T
    qlty5=np.reshape(valuation(aug[:,:-1],tr,trlt),(-1,1))   #line 272
    new5=np.append(new4,qlty5,axis=1)
    print('class=',i,'shape(new5)=',np.shape(new5),end='\t')
    print('mx=',np.max(new5[:,1:-7]),end='\t')
    print('mn=',np.min(new5[:,1:-7]))
    pd.DataFrame(new5[:,:]).to_csv('{}/AugEvl/AugEvl{}{}.csv'.format(cwd,cls,mthd))
    print()

In [7]:
for i in range (1,cls_num+1):                           #تعریف آرایه ی پنجره های کلاس ها
    vars()['ecg_aug'+str(i)+'00']=np.empty((0,smpl_rte+1), float) #Scale    
    vars()['ecg_aug'+str(i)+'10']=np.empty((0,smpl_rte+1), float) #Scale
    vars()['ecg_aug'+str(i)+'20']=np.empty((0,smpl_rte+1), float) #Mag Warp
    vars()['ecg_aug'+str(i)+'30']=np.empty((0,smpl_rte+1), float) #TimWarp
    vars()['ecg_aug'+str(i)+'40']=np.empty((0,smpl_rte+1), float) #GDO
    vars()['ecg_aug'+str(i)+'50']=np.empty((0,smpl_rte+1), float) #frequence
    vars()['ecg_aug'+str(i)+'60']=np.empty((0,smpl_rte+1), float) #GAN
    vars()['ecg_aug'+str(i)+'61']=np.empty((0,smpl_rte+1), float)
    vars()['ecg_aug'+str(i)+'62']=np.empty((0,smpl_rte+1), float)
    vars()['ecg_aug'+str(i)+'70']=np.empty((0,smpl_rte+1), float) #LSTM
    vars()['ecg_aug'+str(i)+'71']=np.empty((0,smpl_rte+1), float)
    vars()['ecg_aug'+str(i)+'72']=np.empty((0,smpl_rte+1), float)

In [8]:
for i in range (1,cls_num+1):                           #تعریف آرایه ی پنجره های کلاس ها
    vlum_sig=10*mx_wndws-len(vars()['wndws'+str(i)])
    
    vars()['ecg_aug'+str(i)+'10']=np.append(vars()['ecg_aug'+str(i)+'10'],sclRaw(ecg,i,int(vlum_sig),mag=.2),axis=0)#0.1
    addqlty(base_wn,vars()['ecg_aug'+str(i)+'10'],10,i)
    print(str(i)+'10 =>',len(vars()['ecg_aug'+str(i)+'10']))              #scl_rndm_beat=Rndm  ,  sclRaw=Mrgnl
    
    vars()['ecg_aug'+str(i)+'20']=np.append(vars()['ecg_aug'+str(i)+'20'],MgWrpRaw(ecg,i,int(vlum_sig),mag=.05),axis=0)
    addqlty(base_wn,vars()['ecg_aug'+str(i)+'20'],20,i)
    print(str(i)+'20 =>',len(vars()['ecg_aug'+str(i)+'20']))
    
    vars()['ecg_aug'+str(i)+'30']=np.append(vars()['ecg_aug'+str(i)+'30'],TimWrp_rndm_beat(ecg,i,int(vlum_sig),mag=.05),axis=0)#.1
    addqlty(base_wn,vars()['ecg_aug'+str(i)+'30'],30,i)
    print(str(i)+'30 =>',len(vars()['ecg_aug'+str(i)+'30']))
    
    vars()['ecg_aug'+str(i)+'40']=np.append(vars()['ecg_aug'+str(i)+'40'],GDO_rndm_beat(ecg,i,int(vlum_sig),mag=.2),axis=0)
    addqlty(base_wn,vars()['ecg_aug'+str(i)+'40'],40,i)
    print(str(i)+'40 =>',len(vars()['ecg_aug'+str(i)+'40']))
    
    vars()['ecg_aug'+str(i)+'50']=np.append(vars()['ecg_aug'+str(i)+'50'],frqnc_rndm_beat(furir_windws,i,int(vlum_sig),mag=.05),axis=0)
    addqlty(base_wn,vars()['ecg_aug'+str(i)+'50'],50,i)
    print(str(i)+'50 =>',len(vars()['ecg_aug'+str(i)+'50']))

class= 1 shape(new5)= (183, 486)	mx= 3.12109375	mn= -3.47265625

110 = 183


C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


class= 1 shape(new5)= (183, 486)	mx= 2.553289331401575	mn= -2.9506748589482243

120 = 183
class= 1 shape(new5)= (182, 486)	mx= 2.3540005706855394	mn= -2.769204794062455

130 = 182
class= 1 shape(new5)= (182, 486)	mx= 3.232048723019061	mn= -4.632422068809412

140 = 182


C:\Users\user\Thesis\Classifying\CNNsplit\CNN_Art_MixedShapesSmallTrain_91_84\augment.py:658: ComplexWarning: Casting complex values to real discards the imaginary part
  p_wn = np.float16(np.concatenate((P,pl), axis=1))


class= 1 shape(new5)= (182, 486)	mx= 2.400390625	mn= -2.810546875

150 = 182
class= 2 shape(new5)= (183, 486)	mx= 2.90234375	mn= -3.974609375

210 = 183
class= 2 shape(new5)= (183, 486)	mx= 2.1959661371020256	mn= -3.1973120314341736

220 = 183
class= 2 shape(new5)= (182, 486)	mx= 1.94754429542485	mn= -2.8906712090286817

230 = 182
class= 2 shape(new5)= (182, 486)	mx= 2.9265819723353084	mn= -4.180428670219246

240 = 182
class= 2 shape(new5)= (182, 486)	mx= 1.994140625	mn= -2.99609375

250 = 182
class= 3 shape(new5)= (183, 486)	mx= 3.677734375	mn= -2.923828125

310 = 183
class= 3 shape(new5)= (183, 486)	mx= 2.6603036244038307	mn= -2.601461779805288

320 = 183
class= 3 shape(new5)= (182, 486)	mx= 2.558506042703387	mn= -2.471061377939925

330 = 182
class= 3 shape(new5)= (182, 486)	mx= 3.94006307839642	mn= -3.9847275675956064

340 = 182
class= 3 shape(new5)= (182, 486)	mx= 2.60546875	mn= -2.517578125

350 = 182
class= 4 shape(new5)= (183, 486)	mx= 3.83984375	mn= -3.935546875

410 = 183
clas